Mini EDA de sitios donde se guardan los archivos adjuntos

In [1]:
import boto3
import pyarrow
import numpy as np
import pandas as pd

In [2]:
f = f's3://inai-summerofdata/parquet/inai.parquet'
df = pd.read_parquet(f, engine='pyarrow')
df.head()

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,...,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012/01/01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
1,1857500002012,2012/01/01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...
2,0001800000712,2012/01/01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0,None
3,0000600003712,2012/01/01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0,INIDOS SECRETARIA DE HACIENDA Y CREDITO PUBLI...
4,0001500002812,2012/01/01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0,None


### Extracción del dominio del archivo de respuesta

In [3]:
my_df = df[~df['ARCHIVORESPUESTA'].isnull()]
#my_df.head()
my_df['site_url'] = my_df['ARCHIVORESPUESTA']
#my_df.head()
my_df.loc[my_df["site_url"].str.contains('infomex.org.mx', case=False, na=None), "site_url"] = 'infomex.org.mx'
my_df.head()
#df['ARCHIVORESPUESTA'].str.extract()

<ipython-input-3-89352743136d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['site_url'] = my_df['ARCHIVORESPUESTA']
/home/rafaelortega/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,...,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,site_url
0,1857500001912,2012/01/01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...,infomex.org.mx
1,1857500002012,2012/01/01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION Oficio Fecha 0...,infomex.org.mx
2,0001800000712,2012/01/01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0,None,infomex.org.mx
3,0000600003712,2012/01/01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0,INIDOS SECRETARIA DE HACIENDA Y CREDITO PUBLI...,infomex.org.mx
4,0001500002812,2012/01/01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,...,2012/02/01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0,None,infomex.org.mx


In [4]:
my_df["site_url"].unique()

array(['infomex.org.mx', 'ADJUNTO'], dtype=object)

Todos los archivos se encuentran en una liga de infomex.org.mx, por lo que no tendría sentido hacer un análisis adicional al que se hizo con url's